<a href="https://colab.research.google.com/github/utkarsh599/Recomendation-sysy/blob/Abhijeet/Leraning_spark4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [58]:
!pip install py4j

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [0]:
import pandas as pd

In [61]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [62]:
GB_category_id=pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/GB_category_id.json")
GBcomments=pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/GBcomments.csv",error_bad_lines=False)
GBvideos=pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/GBvideos.csv",error_bad_lines=False)
US_category=pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/US_category_id.json")
UScomments=pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/UScomments.csv",error_bad_lines=False)
USvideos=pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/USvideos.csv",error_bad_lines=False)


b'Skipping line 113225: expected 4 fields, saw 5\n'
b'Skipping line 158379: expected 4 fields, saw 7\nSkipping line 241590: expected 4 fields, saw 5\nSkipping line 245637: expected 4 fields, saw 7\n'
b'Skipping line 521402: expected 4 fields, saw 5\n'
b'Skipping line 2398: expected 11 fields, saw 21\nSkipping line 2797: expected 11 fields, saw 21\n'
b'Skipping line 41589: expected 4 fields, saw 11\nSkipping line 51628: expected 4 fields, saw 7\nSkipping line 114465: expected 4 fields, saw 5\n'
b'Skipping line 142496: expected 4 fields, saw 8\nSkipping line 189732: expected 4 fields, saw 6\nSkipping line 245218: expected 4 fields, saw 7\n'
b'Skipping line 388430: expected 4 fields, saw 5\n'
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
b'Skipping line 2401: expected 11 fields, saw 21\nSkippi

In [0]:
Comments=pd.concat([GBcomments,UScomments],ignore_index=True)
Videos=pd.concat([GBvideos,USvideos],ignore_index=True)

In [0]:
Comments.comment_text=Comments.comment_text.astype(str)
Comments.likes=Comments.likes.astype(str)
Comments.replies=Comments.replies.astype(str)

In [0]:
Comments['comment_text']=Comments['comment_text'].apply(lambda x: x.lower())

In [66]:
Comments.head(5)

,video_id,comment_text,likes,replies
0,jt2OHQh0HoQ,it's more accurate to call it the m+ (1000) be...,0,0
1,jt2OHQh0HoQ,to be there with a samsung phone\n😂😂😂,1,0
2,jt2OHQh0HoQ,"thank gosh, a place i can watch it without hav...",0,0
3,jt2OHQh0HoQ,what happened to the home button on the iphone...,0,0
4,jt2OHQh0HoQ,power is the disease. care is the cure. keep...,0,0


In [0]:
from pyspark.sql.types import *

In [0]:
schema=StructType([
    StructField("video_id", IntegerType(), True),
    StructField("comment_text", StringType(), True),
    StructField("likes", IntegerType(), True),
    StructField("replies", IntegerType(), True)])

In [0]:
import findspark
findspark.init("spark-2.4.4-bin-hadoop2.7")# SPARK_HOME

from pyspark.sql import SparkSession

In [0]:
#spark = SparkSession.builder.master("local[*]").appName('Micro-Video-Recommandation-System').getOrCreate()

In [0]:
#!pip install pyspark

In [72]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext

conf = SparkConf().setAppName("Pyspark basics")
#sc = SparkContext(conf=conf)
sqlCtx = SQLContext(sc)

sdf=sqlCtx.createDataFrame(Comments) 

sdf.show(5)



+-----------+--------------------+-----+-------+
|   video_id|        comment_text|likes|replies|
+-----------+--------------------+-----+-------+
|jt2OHQh0HoQ|it's more accurat...|    0|      0|
|jt2OHQh0HoQ|to be there with ...|    1|      0|
|jt2OHQh0HoQ|thank gosh, a pla...|    0|      0|
|jt2OHQh0HoQ|what happened to ...|    0|      0|
|jt2OHQh0HoQ|power is the dise...|    0|      0|
+-----------+--------------------+-----+-------+
only showing top 5 rows



In [0]:
comments_rdd = sdf.select('comment_text').rdd.flatMap(lambda x: x)
#comments_rdd.collect()

In [0]:
header = comments_rdd.first()

In [0]:
data_rmv_col = comments_rdd.filter(lambda row: row != header)

In [0]:
lowerCase_sentRDD = data_rmv_col.map(lambda x : x.lower())

In [0]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [0]:
def sent_TokenizeFunct(x):
    doc=nlp(x)
    tokens=[]
    for token in doc:
      if((token.is_stop==False and token.is_punct==False) and token.is_alpha==True):
        tokens.append(token.lemma_)
    return ' '.join(tokens)
    
sentenceTokenizeRDD = lowerCase_sentRDD.map(sent_TokenizeFunct)

In [87]:
sentenceTokenizeRDD.first()

'samsung'

In [88]:
sentenceTokenizeRDD.collect()

IllegalArgumentException: ignored